# Gender Classification from Text Blog

## Load dataset into notebook

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from bs4 import BeautifulSoup
import string
import nltk
nltk.download()
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

import re


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn import metrics

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hoangvo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/hoangvo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/hoangvo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#!  conda install  openpyxl

In [3]:

#df = pd.read_excel('blog-gender-dataset/blog-gender-dataset.xlsx', engine='openpyxl')
headers = ["text", "class"]
text_df = pd.read_csv('blog-gender-dataset/blog-gender-dataset.csv')

In [4]:
text_df = text_df.iloc[:, [0, 1]]
text_df.columns = headers
text_df

,text,class
0,Guest Demo: Eric Iverson’s Itty Bitty Search\...,M
1,Who moved my Cheese??? The world has been de...,M
2,Yesterday I attended a biweekly meeting of an...,M
3,Liam is nothing like Natalie. Natalie never w...,F
4,"In the EU we have browser choice, but few know...",M
...,...,...
3226,It was a scavenger style race with checkpoints...,M
3227,Finally! I got a full day's work done. Almost ...,F
3228,"At the height of laughter, the universe is flu...",M
3229,"I like birds, especially woodpeckers and MOST ...",M


In [5]:
text_df.shape

(3231, 2)

In [6]:
text_df["class"].describe()

count     3226
unique       8
top          M
freq      1546
Name: class, dtype: object

In [7]:
text_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3231 entries, 0 to 3230
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    3224 non-null   object
 1   class   3226 non-null   object
dtypes: object(2)
memory usage: 50.6+ KB


In [8]:
text_df[text_df['class']=="M"].shape

(1546, 2)

In [9]:
text_df[text_df['class']=="F"].shape

(1390, 2)

In [10]:
text_df = text_df.dropna()
text_df

,text,class
0,Guest Demo: Eric Iverson’s Itty Bitty Search\...,M
1,Who moved my Cheese??? The world has been de...,M
2,Yesterday I attended a biweekly meeting of an...,M
3,Liam is nothing like Natalie. Natalie never w...,F
4,"In the EU we have browser choice, but few know...",M
...,...,...
3226,It was a scavenger style race with checkpoints...,M
3227,Finally! I got a full day's work done. Almost ...,F
3228,"At the height of laughter, the universe is flu...",M
3229,"I like birds, especially woodpeckers and MOST ...",M


In [11]:
text_df["class"] = text_df["class"].replace(' F', 'F')
text_df["class"] = text_df["class"].replace('f', 'F')
text_df["class"] = text_df["class"].replace('F ', 'F')
text_df["class"] = text_df["class"].replace(' M', 'M')
text_df["class"] = text_df["class"].replace('m', 'M')
text_df["class"] = text_df["class"].replace(' M ', 'M')
text_df['class'].value_counts()

/Users/hoangvo/anaconda3/envs/Data-mining/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/hoangvo/anaconda3/envs/Data-mining/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/hoangvo/anaconda3/envs/Data-mining/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

M    1677
F    1547
Name: class, dtype: int64

In [12]:
text_df2 = pd.read_json('data/train.json')

In [13]:
text_df2 = text_df2[["post","gender"]]
text_df2.columns = headers
text_df2["class"] = text_df2["class"].replace('female', 'F')
text_df2["class"] = text_df2["class"].replace('male', 'M')
text_df2 = text_df2[text_df2["text"]!= ""]

In [14]:
#df_M = [text_df1[text_df1["class"]=="M"], text_df2[text_df2["class"]=="M"]]
df_M = [text_df2[text_df2["class"]=="M"]]
df_M = pd.concat(df_M)
df_M = df_M.sample(frac=1).reset_index(drop=True)
#df_F = [text_df1[text_df1["class"]=="F"], text_df2[text_df2["class"]=="F"]]
df_F = [text_df2[text_df2["class"]=="F"]]
df_F = pd.concat(df_F)
df_F = df_F.sample(frac=1).reset_index(drop=True)

In [15]:
#text_df = [text_df, df_M[:400], df_F[:400]]
text_df = [ df_M[:100000], df_F[:100000]]
text_df = pd.concat(text_df)

In [16]:
text_df = text_df.sample(frac=1).reset_index(drop=True)

In [17]:
text_df = text_df.dropna()
text_df = text_df.sample(frac=1).reset_index(drop=True)

## Clean text

In [18]:
#import nltk
#nltk.download()

https://towardsdatascience.com/nlp-for-beginners-cleaning-preprocessing-text-data-ae8e306bef0f

In [19]:
text_df["text"][1]

"Hmmmm, I haven't done much this week. Me and Britt went to the Mall of Georgia on monday, yesterday I did nothing, and today, me and my mom went out to eat and then we went out to dinner for my dad's birthday at Macaroni Grill. Tomorrow will be busy. Registration at 8:30, then FINALLY I get my license at 10:30. I hate to admit that I'm almost 18 and don't have my license, so after tomorrow I am FREE. Then, tomorrow night, I am either going to see Blues Traveler for free at the Variety Playhouse with Britt or to see Bob Margolin at Blind Willie's with my dad. I'd rather see Margolin again because he is a badass. Oh well, that's all. Ron Isley is the biggest pimp ever."

In [20]:
#remove html
def remove_html(text):
    soup = BeautifulSoup(text, 'lxml')
    html_free = soup.get_text()
    return html_free

#remove punctuation
def remove_punctuation(text):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = " ".join([c for c in text if c not in punctuations])
    return no_punct
#textclean = text_df['text'].apply(lambda x : remove_punctuation(x))
#instantiate tokenizer
tokenizer = RegexpTokenizer(r'\w+')
#textclean = textclean.apply(lambda x)

In [21]:

def  clean_text(df, text_field, new_text_field_name):
    df[new_text_field_name] = df[text_field].str.lower()
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    # remove numbers
    df[new_text_field_name] = df[new_text_field_name].apply(lambda elem: re.sub(r"\d+", "", elem))
    
    return df
data_clean = clean_text(text_df, 'text', 'text_clean')
data_clean.head()

,text,class,text_clean
0,I just got word that I was not cast in the Law...,M,i just got word that i was not cast in the law...
1,"Hmmmm, I haven't done much this week. Me and B...",M,hmmmm i havent done much this week me and brit...
2,i'm not even sure what memorial day is suppose...,F,im not even sure what memorial day is supposed...
3,"ok, so we are having a classic book crisis at ...",F,ok so we are having a classic book crisis at o...
4,urlLink Steve Newton talking to Laura at hi...,F,urllink steve newton talking to laura at hi...


In [22]:
#import nltk.corpus
#nltk.download('stopwords')
#from nltk.corpus import stopwords
stop = stopwords.words('english')
data_clean['text_clean'] = data_clean['text_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
data_clean.head()

,text,class,text_clean
0,I just got word that I was not cast in the Law...,M,got word cast law order episode good news cast...
1,"Hmmmm, I haven't done much this week. Me and B...",M,hmmmm havent done much week britt went mall ge...
2,i'm not even sure what memorial day is suppose...,F,im even sure memorial day supposed celebrate v...
3,"ok, so we are having a classic book crisis at ...",F,ok classic book crisis housewe cannot find har...
4,urlLink Steve Newton talking to Laura at hi...,F,urllink steve newton talking laura party urllink


In [23]:
data_clean['text_clean'][0]

'got word cast law order episode good news casting director show called manager got audition first place give feedback said wasnt director mind role gave good read probably law order future besides cast could ask ive game ten days im pretty happy far'

In [24]:
#import nltk 
#nltk.download('punkt')
#from nltk.tokenize import sent_tokenize, word_tokenize

data_clean['text_tokens'] = data_clean['text_clean'].apply(lambda x: word_tokenize(x))

def word_stemmer(text):
    stem_text = [PorterStemmer().stem(i) for i in text]
    return stem_text

data_clean['text_tokens_stem'] = data_clean['text_tokens'].apply(lambda x: word_stemmer(x))
data_clean.head()

,text,class,text_clean,text_tokens,text_tokens_stem
0,I just got word that I was not cast in the Law...,M,got word cast law order episode good news cast...,"[got, word, cast, law, order, episode, good, n...","[got, word, cast, law, order, episod, good, ne..."
1,"Hmmmm, I haven't done much this week. Me and B...",M,hmmmm havent done much week britt went mall ge...,"[hmmmm, havent, done, much, week, britt, went,...","[hmmmm, havent, done, much, week, britt, went,..."
2,i'm not even sure what memorial day is suppose...,F,im even sure memorial day supposed celebrate v...,"[im, even, sure, memorial, day, supposed, cele...","[im, even, sure, memori, day, suppos, celebr, ..."
3,"ok, so we are having a classic book crisis at ...",F,ok classic book crisis housewe cannot find har...,"[ok, classic, book, crisis, housewe, can, not,...","[ok, classic, book, crisi, housew, can, not, f..."
4,urlLink Steve Newton talking to Laura at hi...,F,urllink steve newton talking laura party urllink,"[urllink, steve, newton, talking, laura, party...","[urllink, steve, newton, talk, laura, parti, u..."


In [25]:
#nltk.download('wordnet')
#from nltk.stem import WordNetLemmatizer

def word_lemmatizer(text):
    lem_text = [WordNetLemmatizer().lemmatize(i) for i in text]
    return lem_text

data_clean['text_tokens_lemma'] = data_clean['text_tokens'].apply(lambda x: word_lemmatizer(x))
data_clean.head()

,text,class,text_clean,text_tokens,text_tokens_stem,text_tokens_lemma
0,I just got word that I was not cast in the Law...,M,got word cast law order episode good news cast...,"[got, word, cast, law, order, episode, good, n...","[got, word, cast, law, order, episod, good, ne...","[got, word, cast, law, order, episode, good, n..."
1,"Hmmmm, I haven't done much this week. Me and B...",M,hmmmm havent done much week britt went mall ge...,"[hmmmm, havent, done, much, week, britt, went,...","[hmmmm, havent, done, much, week, britt, went,...","[hmmmm, havent, done, much, week, britt, went,..."
2,i'm not even sure what memorial day is suppose...,F,im even sure memorial day supposed celebrate v...,"[im, even, sure, memorial, day, supposed, cele...","[im, even, sure, memori, day, suppos, celebr, ...","[im, even, sure, memorial, day, supposed, cele..."
3,"ok, so we are having a classic book crisis at ...",F,ok classic book crisis housewe cannot find har...,"[ok, classic, book, crisis, housewe, can, not,...","[ok, classic, book, crisi, housew, can, not, f...","[ok, classic, book, crisis, housewe, can, not,..."
4,urlLink Steve Newton talking to Laura at hi...,F,urllink steve newton talking laura party urllink,"[urllink, steve, newton, talking, laura, party...","[urllink, steve, newton, talk, laura, parti, u...","[urllink, steve, newton, talking, laura, party..."


In [26]:
#part of speech tagging and chunking
def word_pos_tagger(text):
    pos_tagged_text = nltk.pos_tag(text)
    return pos_tagged_text

nltk.download('averaged_perceptron_tagger')
data_clean['text_tokens_pos_tagged'] = data_clean['text_tokens'].apply(lambda x: word_pos_tagger(x))
data_clean.head()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/hoangvo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,text,class,text_clean,text_tokens,text_tokens_stem,text_tokens_lemma,text_tokens_pos_tagged
0,I just got word that I was not cast in the Law...,M,got word cast law order episode good news cast...,"[got, word, cast, law, order, episode, good, n...","[got, word, cast, law, order, episod, good, ne...","[got, word, cast, law, order, episode, good, n...","[(got, VBD), (word, NN), (cast, NN), (law, NN)..."
1,"Hmmmm, I haven't done much this week. Me and B...",M,hmmmm havent done much week britt went mall ge...,"[hmmmm, havent, done, much, week, britt, went,...","[hmmmm, havent, done, much, week, britt, went,...","[hmmmm, havent, done, much, week, britt, went,...","[(hmmmm, NN), (havent, NN), (done, VBN), (much..."
2,i'm not even sure what memorial day is suppose...,F,im even sure memorial day supposed celebrate v...,"[im, even, sure, memorial, day, supposed, cele...","[im, even, sure, memori, day, suppos, celebr, ...","[im, even, sure, memorial, day, supposed, cele...","[(im, RB), (even, RB), (sure, JJ), (memorial, ..."
3,"ok, so we are having a classic book crisis at ...",F,ok classic book crisis housewe cannot find har...,"[ok, classic, book, crisis, housewe, can, not,...","[ok, classic, book, crisi, housew, can, not, f...","[ok, classic, book, crisis, housewe, can, not,...","[(ok, JJ), (classic, JJ), (book, NN), (crisis,..."
4,urlLink Steve Newton talking to Laura at hi...,F,urllink steve newton talking laura party urllink,"[urllink, steve, newton, talking, laura, party...","[urllink, steve, newton, talk, laura, parti, u...","[urllink, steve, newton, talking, laura, party...","[(urllink, JJ), (steve, VBP), (newton, NN), (t..."


In [27]:
# Remove short words less than 3
data_clean['fine'] = data_clean['text_tokens_lemma'].apply(lambda x: ' '.join([w for w in x if len(w)>3]))
# Count the length of characters
data_clean['clean_length'] = data_clean['fine'].apply(len)
# Remove rows where character length <= 5
data_clean = data_clean[data_clean.clean_length > 5]
data_clean.sample(n=15)


,text,class,text_clean,text_tokens,text_tokens_stem,text_tokens_lemma,text_tokens_pos_tagged,fine,clean_length
15919,"Today started work at 11am, so got to wake up ...",M,today started work got wake late tis morning t...,"[today, started, work, got, wake, late, tis, m...","[today, start, work, got, wake, late, ti, morn...","[today, started, work, got, wake, late, ti, mo...","[(today, NN), (started, VBD), (work, NN), (got...",today started work wake late morning might suz...,1241
28138,urlLink LETS PLAY POCKER!!!!!!!!!!! Y DO GI...,F,urllink lets play pocker girls never know play...,"[urllink, lets, play, pocker, girls, never, kn...","[urllink, let, play, pocker, girl, never, know...","[urllink, let, play, pocker, girl, never, know...","[(urllink, JJ), (lets, NNS), (play, VBP), (poc...",urllink play pocker girl never know play pocke...,55
55266,(At Blockbuster) Austin: Is that Cate Blanchet...,F,blockbuster austin cate blanchett getting sara...,"[blockbuster, austin, cate, blanchett, getting...","[blockbust, austin, cate, blanchett, get, sara...","[blockbuster, austin, cate, blanchett, getting...","[(blockbuster, NN), (austin, NN), (cate, NN), ...",blockbuster austin cate blanchett getting sara...,191
172632,Hello everybody. I kind of forgot about the bl...,M,hello everybody kind forgot blog today miss do...,"[hello, everybody, kind, forgot, blog, today, ...","[hello, everybodi, kind, forgot, blog, today, ...","[hello, everybody, kind, forgot, blog, today, ...","[(hello, NN), (everybody, NN), (kind, NN), (fo...",hello everybody kind forgot blog today miss do...,108
91488,So I'm thinking about all the things that I th...,F,im thinking things think wrong family know im ...,"[im, thinking, things, think, wrong, family, k...","[im, think, thing, think, wrong, famili, know,...","[im, thinking, thing, think, wrong, family, kn...","[(im, NN), (thinking, VBG), (things, NNS), (th...",thinking thing think wrong family know faultle...,1779
43531,Wowsers. Lots and lots and lots of stuffs hap...,F,wowsers lots lots lots stuffs happened friday ...,"[wowsers, lots, lots, lots, stuffs, happened, ...","[wowser, lot, lot, lot, stuff, happen, friday,...","[wowsers, lot, lot, lot, stuff, happened, frid...","[(wowsers, NNS), (lots, VBP), (lots, NNS), (lo...",wowsers stuff happened friday finished star te...,1675
25096,"Hey ya'll, I just got back from a week-long v...",F,hey yall got back weeklong vacation switzerlan...,"[hey, yall, got, back, weeklong, vacation, swi...","[hey, yall, got, back, weeklong, vacat, switze...","[hey, yall, got, back, weeklong, vacation, swi...","[(hey, NN), (yall, NN), (got, VBD), (back, RB)...",yall back weeklong vacation switzerland found ...,497
137844,HEY! Christmas was of most fun! I think as you...,M,hey christmas fun think get older get realize ...,"[hey, christmas, fun, think, get, older, get, ...","[hey, christma, fun, think, get, older, get, r...","[hey, christmas, fun, think, get, older, get, ...","[(hey, NN), (christmas, VBP), (fun, NN), (thin...",christmas think older realize christmas thats ...,964
79339,Fuck this urlLink guy . His thinks that Denn...,M,fuck urllink guy thinks dennis kucinich should...,"[fuck, urllink, guy, thinks, dennis, kucinich,...","[fuck, urllink, guy, think, denni, kucinich, s...","[fuck, urllink, guy, think, dennis, kucinich, ...","[(fuck, JJ), (urllink, JJ), (guy, NN), (thinks...",fuck urllink think dennis kucinich shouldnt co...,220
30904,"When I was younger I thought I was invincible,...",F,younger thought invincible nothing couldnt rem...,"[younger, thought, invincible, nothing, couldn...","[younger, thought, invinc, noth, couldnt, reme...","[younger, thought, invincible, nothing, couldn...","[(younger, JJR), (thought, VBD), (invincible, ...",younger thought invincible nothing couldnt rem...,1094


In [28]:
data_clean.shape

(198645, 9)

In [29]:
def preprocess_sample(text):
    #clean text
    newText = text.str.lower()
    newText = newText.apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))
    newText = newText.apply(lambda elem: re.sub(r"\d+", "", elem))
    #stopwords
    from nltk.corpus import stopwords
    stop = stopwords.words('english')
    newText = newText.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    #stemmer
    newText = newText.apply(lambda x: word_tokenize(x))
    newText_stem = newText.apply(lambda x: word_stemmer(x))
    #token lemma
    newText_lem = newText.apply(lambda x: word_lemmatizer(x))
    newText_tag = newText.apply(lambda x: word_pos_tagger(x))
 


## Declare X and Y data

https://pub.towardsai.net/does-a-machine-know-your-gender-based-on-your-tweets-43b14740fd54

In [30]:
# Import libraries
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import f1_score
# Bag-of-words features
bow_vectorizer = CountVectorizer(stop_words='english')
# Bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(data_clean['fine'])
df_bow = pd.DataFrame(bow.todense(), columns=bow_vectorizer.get_feature_names())
df_bow

,aaaa,aaaaa,aaaaaa,aaaaaaa,aaaaaaaa,aaaaaaaaa,aaaaaaaaaaa,aaaaaaaaaaaa,aaaaaaaaaaaaa,aaaaaaaaaaaaaa,...,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz,zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198640,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
198641,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
198642,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
198643,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
data_clean['gender'] = data_clean['class'].apply(lambda x: 1 if x=='F' else 0)
# Splitting the data into training and test set
X = df_bow
y = data_clean['gender']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
# Fitting on Logistic Regression model
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
prediction_bow = logreg.predict_proba(X_test)
# Calculating the F1 score
# If prediction is greater than or equal to 0.5 than 1, else 0
# Gender, 0 = male and 1 = female
prediction_int = prediction_bow[:,1]>=0.5
prediction_int = prediction_int.astype(np.int)
# Calculating F1 score
log_bow = f1_score(y_test, prediction_int)
log_bow

In [ ]:
predictions = logreg.predict(X_test)

In [ ]:
predictions

In [ ]:
# Use score method to get accuracy of model
score = logreg.score(X_test, y_test)
print(score)

In [ ]:


cm = metrics.confusion_matrix(y_test, predictions)
print(cm)

https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a

https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, y_train)

In [ ]:
predicted = clf.predict(X_test)
np.mean(predicted == y_test)

In [ ]:
#from sklearn.pipeline import Pipeline
#from sklearn.feature_extraction.text import TfidfTransformer
#text_clf = Pipeline([('vect', CountVectorizer()),
#                      ('tfidf', TfidfTransformer()),
#                      ('clf', MultinomialNB()),
 #                      ])
#text_clf = text_clf.fit(X_train, y_train)

In [ ]:
#predicted = text_clf.predict(X_test)
#np.mean(predicted == y_test)

In [ ]:
from sklearn.linear_model import SGDClassifier
clf_svm = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter_no_change=5, random_state=42)
_ = clf_svm.fit(X_train, y_train)
predicted_svm = clf_svm.predict(X_test)
np.mean(predicted_svm == y_test)

In [ ]:
X1 = text_df["text"]
y1 = text_df["class"].apply(lambda x: 1 if x=='F' else 0)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X1, y1, test_size=0.20)

https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a

In [ ]:

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
stemmed_count_vect = StemmedCountVectorizer(stop_words='english')
text_mnb_stemmed = Pipeline([('vect', stemmed_count_vect),
                      ('tfidf', TfidfTransformer()),
                      ('mnb', MultinomialNB(fit_prior=False)),
 ])
text_mnb_stemmed = text_mnb_stemmed.fit(X_train2, y_train2)
predicted_mnb_stemmed = text_mnb_stemmed.predict(X_test2)
np.mean(predicted_mnb_stemmed == y_test2)

Apply Neuron Network: https://realpython.com/python-keras-text-classification/